In [3]:
import sys
print(sys.executable)
print(sys.path)

/Users/thomasmanson/.pyenv/versions/3.10.6/envs/recettes-et-sentiments/bin/python
['/Users/thomasmanson/code/mansonthomas-wagon/projets/recettes_et_sentiments/eda_notebooks', '/Users/thomasmanson/code/mansonthomas-wagon/projets/recettes_et_sentiments/eda_notebooks', '/Users/thomasmanson/code/mansonthomas-wagon/04-Decision-Science/01-Project-Setup/data-context-and-setup', '/Users/thomasmanson/.pyenv/versions/3.10.6/lib/python310.zip', '/Users/thomasmanson/.pyenv/versions/3.10.6/lib/python3.10', '/Users/thomasmanson/.pyenv/versions/3.10.6/lib/python3.10/lib-dynload', '', '/Users/thomasmanson/.pyenv/versions/3.10.6/envs/recettes-et-sentiments/lib/python3.10/site-packages', '/Users/thomasmanson/code/mansonthomas-wagon/projets/recettes_et_sentiments/recettes_et_sentiments']


In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import ast

pd.set_option('display.max_rows', 100)

from recettes_et_sentiments.api_model import data, preprocessing


ModuleNotFoundError: No module named 'recettes_et_sentiments'

In [2]:
!pip list | grep recettes


recettes-et-sentiments            0.1         /Users/thomasmanson/code/mansonthomas-wagon/projets/recettes_et_sentiments/recettes_et_sentiments


In [4]:
df = pd.read_csv("../../batch-1672-recettes-et-sentiments-data/RAW_recipes.csv",
                 parse_dates=['submitted'],
                 engine='python')



In [6]:
df.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [8]:
df.dtypes


name                      object
id                         int64
minutes                    int64
contributor_id             int64
submitted         datetime64[ns]
tags                      object
nutrition                 object
n_steps                    int64
steps                     object
description               object
ingredients               object
n_ingredients              int64
dtype: object

In [10]:
def convert_column_to_list(column):
    return column.apply(ast.literal_eval)


In [11]:
df['tags'] = convert_column_to_list(df['tags'])
df['nutrition'] = convert_column_to_list(df['nutrition'])
df['steps'] = convert_column_to_list(df['steps'])
df['ingredients'] = convert_column_to_list(df['ingredients'])

In [12]:
nutrition_columns = ['calories', 'total_fat_pdv', 'sugar_pdv', 'sodium_pdv', 'protein_pdv', 'saturated_fat_pdv', 'carbohydrates_pdv']
df[nutrition_columns] = pd.DataFrame(df['nutrition'].tolist(), index=df.index)

In [5]:

df[df[['name']].duplicated('name', keep=False)].sort_values('name').head(50)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
846,10 bean soup,470575,180,1020526,2011-12-26,"['time-to-make', 'course', 'main-ingredient', ...","[228.0, 10.0, 38.0, 47.0, 42.0, 8.0, 6.0]",9,"['finely chop onion , red and green bell peppe...",we made this without the smoked turkey and ham...,"['beans', 'onion', 'red bell pepper', 'green b...",13
806,10 bean soup,348965,780,215956,2009-01-11,"['course', 'main-ingredient', 'soups-stews', '...","[63.1, 3.0, 10.0, 11.0, 6.0, 1.0, 3.0]",11,"['soak beans in water overnight', 'drain the b...","yes, another soup recipe! we really love a ho...","['10 bean soup mix', 'fat-free low-sodium chic...",14
1314,3 bean baked beans,313237,40,407338,2008-07-11,"['60-minutes-or-less', 'time-to-make', 'course...","[329.9, 26.0, 48.0, 39.0, 33.0, 31.0, 9.0]",10,"['fry bacon until crisp , reserving 2 teaspoon...","great for cookouts, family gatherings, potluck...","['bacon', 'ground beef', 'onion', 'tomato sauc...",12
1200,3 bean baked beans,187438,80,208455,2006-09-24,"['time-to-make', 'course', 'main-ingredient', ...","[574.8, 35.0, 134.0, 48.0, 46.0, 39.0, 23.0]",4,"['in a large bowl combine cooked bacon , hambu...",a wonderful bean dish that always gets gobbled...,"['bacon', 'hamburger', 'onion', 'kidney beans'...",11
1315,3 bean salad,258846,15,607801,2007-10-13,"['15-minutes-or-less', 'time-to-make', 'course...","[502.3, 44.0, 37.0, 41.0, 24.0, 19.0, 16.0]",6,"['in medium bowl combine mustard , vinegar and...",i got this recipe from one of rachel ray's coo...,"['cut green beans', 'dijon mustard', 'sugar', ...",9
1203,3 bean salad,359316,20,1158733,2009-03-05,"['30-minutes-or-less', 'time-to-make', 'course...","[260.6, 14.0, 79.0, 18.0, 13.0, 6.0, 12.0]",4,"['place beans , pepper and onions in a large b...","a cold, refreshing, salad that's quick and easy.","['canned green beans', 'yellow wax beans', 're...",10
1335,3 ingredient peanut butter cookies,323810,13,138435,2008-09-07,"['15-minutes-or-less', 'time-to-make', 'course...","[197.1, 17.0, 74.0, 4.0, 11.0, 11.0, 6.0]",10,"['preheat oven to 350', 'combine the three ing...",this recipe is so simple. sometimes i put cho...,"['peanut butter', 'sugar', 'egg', 'chocolate c...",4
1259,3 ingredient peanut butter cookies,525818,25,1803607183,2016-02-01,"['30-minutes-or-less', 'time-to-make', 'course...","[98.4, 8.0, 37.0, 2.0, 5.0, 5.0, 3.0]",12,"['1', 'mix all three ingredients in a large bo...",peanut butter cookies,"['peanut butter', 'granulated sugar', 'egg']",3
1292,3 pepper cheese spread,351678,65,535633,2009-01-23,"['time-to-make', 'course', 'preparation', 'for...","[88.5, 12.0, 2.0, 4.0, 6.0, 23.0, 0.0]",3,"['mix all ingredients except crackers , cover'...",this recipe comes from the kraft holiday cookb...,"['cream cheese', 'monterey jack and cheddar ch...",6
1343,3 pepper cheese spread,197894,65,262312,2006-11-27,"['time-to-make', 'course', 'main-ingredient', ...","[190.4, 26.0, 3.0, 31.0, 15.0, 52.0, 0.0]",3,"['mix all ingredients', 'refridgerate at least...","an easy, yummy spread to entertain with! you ...","['cream cheese', 'cheddar cheese', 'monterey j...",6
